In [112]:
import tqdm
import random
import pathlib
import itertools
import collections
import keras

import glob
import pandas as pd
import os
import cv2
import numpy as np
import remotezip as rz
import einops
import random
import shutil

import tensorflow as tf

from pathlib import Path

# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed
from keras import layers
import matplotlib.pyplot as plt

In [1]:
#########################
#########################
#########################

# Data Preprocessing

In [276]:
# Create a directory to store all the clips
clips_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips"
os.makedirs(clips_dir, exist_ok=True)

# Loop through all the CSV files
csv_dir = r'C:\Users\BEE Team\Desktop\code\Summarys_labels'
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
csv_files = csv_files[1:]


# Loop through all the CSV files
for csv_file in csv_files:
    if csv_file.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(os.path.join(csv_dir, csv_file))

        # Directory of videos
        video_dir = r'C:\Users\BEE Team\Desktop\Videos\Daniele_Vids_Lab\LD\Yellow'
        df['Video_path'] = df['file'].apply(lambda x: os.path.join(video_dir, x.split('_labels')[0] + '.mp4'))

        # Create a directory to store the clips for this csv file
        csv_clips_dir = os.path.join(clips_dir, os.path.splitext(csv_file.split('_')[2])[0])
        os.makedirs(csv_clips_dir, exist_ok=True)

        # Extract video clips from each row of the dataframe and save them to a file
        for index, row in df.iterrows():
            video_path = row['Video_path']
            start_frame = row['start_frame']
            end_frame = row['end_frame']
            classification = row['classification']
            
            video = cv2.VideoCapture(video_path)
            video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            clip_frames = []
            for i in range(start_frame, end_frame):
                ret, frame = video.read()
                if ret:
                    clip_frames.append(frame)
                else:
                    break
            

            if duration > 120:  # if duration is more than 2 seconds (assuming 60 fps)
                num_clips = int(duration / 120)  # split into 120-frame clips
                for i in range(num_clips + 1):
                    start_idx = i * 120
                    end_idx = min(start_idx + 120, duration)
                    subclip_frames = clip_frames[start_idx:end_idx]
                    subclip_path = os.path.join(csv_clips_dir, f'{classification}_{index}_{i}.mp4')
                    subclip = cv2.VideoWriter(subclip_path, cv2.VideoWriter_fourcc(*'XVID'), 60, (frame.shape[1], frame.shape[0]))
                    for frame in subclip_frames:
                        subclip.write(frame)
                    subclip.release()



            else:  # if duration is less than or equal to 2 seconds
                clip_path = os.path.join(csv_clips_dir, f'{classification}_{index}.mp4')
                clip = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*'XVID'), 60, (frame.shape[1], frame.shape[0]))
                for frame in clip_frames:
                    clip.write(frame)
                clip.release()           
            

In [278]:
# set the root directory of your videos
root_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips"

# set the directories where you want to move the videos to
train_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\train"
test_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test"
val_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\val"


# set the split ratios
train_ratio = 0.6
test_ratio = 0.2
val_ratio = 0.2

# loop through all the subdirectories in the root directory
for subdir in os.listdir(root_dir):
    
    # check if the subdirectory is a directory
    if os.path.isdir(os.path.join(root_dir, subdir)):
        
        # get the list of videos in the subdirectory
        video_list = [f for f in os.listdir(os.path.join(root_dir, subdir)) if f.endswith(".mp4")]
        
        # calculate the number of videos in each split
        num_videos = len(video_list)
        num_train = int(0.6 * num_videos)
        num_test = int(0.2 * num_videos)
        num_val = num_videos - num_train - num_test
        
        # shuffle the video list
        random.shuffle(video_list)
        
        # move the videos to the appropriate split directory
        for i, video in enumerate(video_list):
            
            if i < num_train:
                dest_dir = os.path.join(train_dir, video.split("_")[0])
            elif i < num_train + num_test:
                dest_dir = os.path.join(test_dir, video.split("_")[0])
            else:
                dest_dir = os.path.join(val_dir, video.split("_")[0])
            
            os.makedirs(dest_dir, exist_ok=True)
            src_file = os.path.join(root_dir, subdir, video)
            dest_file = os.path.join(dest_dir, video)
            shutil.move(src_file, dest_file)

In [275]:
# Run only to delete folders

#shutil.rmtree(r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips")

In [180]:
########
########
########

# Classifcation

In [448]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

In [449]:
def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

In [450]:
os.getcwd()

'C:\\Users\\BEE Team\\Desktop\\code'

In [435]:
video_path = r"Summarys_labels\Sep_Clips\val\IntCleaner\IntCleaner_9.mp4"
sample_video = frames_from_video_file(video_path, n_frames = 10)
sample_video.shape

(10, 224, 224, 3)

In [451]:
def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=10)
  return embed.embed_file('./animation.gif')

In [452]:
#to_gif(sample_video)

In [453]:
video_dir = r'Summarys_labels\Sep_Clips\train\IntCleaner'

# get a list of all the video files in the directory
video_files = glob.glob(os.path.join(video_dir, '*.mp4'))

# select the first video file in the list
video_path = video_files[0]

# extract 50 frames from the selected video file
sample_videos = frames_from_video_file(video_path, 50)
#to_gif(sample_videos)

In [454]:
class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.mp4'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [516]:
fg = FrameGenerator(Path('Summarys_labels/Sep_Clips/train'), 10, training=True)

frames,label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (10, 224, 224, 3)
Label: 5


In [517]:
# Create the training set
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(Path('Summarys_labels/Sep_Clips/train'), 10, training=True),
                                          output_signature = output_signature)

In [518]:
for frames, labels in train_ds.take(10):
  print(labels)

tf.Tensor(5, shape=(), dtype=int16)
tf.Tensor(5, shape=(), dtype=int16)
tf.Tensor(5, shape=(), dtype=int16)
tf.Tensor(4, shape=(), dtype=int16)
tf.Tensor(5, shape=(), dtype=int16)
tf.Tensor(5, shape=(), dtype=int16)
tf.Tensor(4, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)


In [519]:
# Create the validation set
val_ds = tf.data.Dataset.from_generator(FrameGenerator(Path('Summarys_labels/Sep_Clips/val'), 10),
                                        output_signature = output_signature)

In [520]:
# Print the shapes of the data
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (10, 224, 224, 3)
Shape of training labels: ()
Shape of validation set of frames: (10, 224, 224, 3)
Shape of validation labels: ()


# Configure the dataset for performance

In [521]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [522]:
train_ds = train_ds.batch(8)
val_ds = val_ds.batch(8)

In [462]:
next(iter(train_ds))

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.
Traceback (most recent call last):

  File "C:\Users\BEE Team\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\ops\script_ops.py", line 267, in __call__
    ret = func(*args)

  File "C:\Users\BEE Team\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\BEE Team\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\BEE Team\AppData\Local\Temp\ipykernel_13092\1321760800.py", line 30, in __call__
    video_frames = frames_from_video_file(path, self.n_frames)

  File "C:\Users\BEE Tea

In [463]:
#train_ds = train_ds.batch(2)
#val_ds = val_ds.batch(2)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (2, 10, 224, 224, 3)
Shape of training labels: (2,)
Shape of validation set of frames: (2, 10, 224, 224, 3)
Shape of validation labels: (2,)


# Next steps

In [686]:
net = tf.keras.applications.EfficientNetB0(include_top = False)
net.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=255),
    tf.keras.layers.TimeDistributed(net),
    tf.keras.layers.Dense(10),
    tf.keras.layers.GlobalAveragePooling3D()
])

model.compile(optimizer = 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

model.fit(train_ds, 
          epochs = 100,
          validation_data = val_ds,
          callbacks = tf.keras.callbacks.EarlyStopping(patience = 25, monitor = 'val_loss'))

Epoch 1/100
12/12 [==============================] - 89s 7s/step - loss: 1.7227 - accuracy: 0.3158 - val_loss: 1.4592 - val_accuracy: 0.4091
Epoch 2/100
12/12 [==============================] - 36s 3s/step - loss: 1.2686 - accuracy: 0.5158 - val_loss: 1.3539 - val_accuracy: 0.3409
Epoch 3/100
12/12 [==============================] - 35s 3s/step - loss: 1.1452 - accuracy: 0.5263 - val_loss: 1.2955 - val_accuracy: 0.3864
Epoch 4/100
12/12 [==============================] - 34s 3s/step - loss: 1.1019 - accuracy: 0.5579 - val_loss: 1.2946 - val_accuracy: 0.3864
Epoch 5/100
12/12 [==============================] - 34s 3s/step - loss: 1.0702 - accuracy: 0.5263 - val_loss: 1.2966 - val_accuracy: 0.4318
Epoch 6/100
12/12 [==============================] - 34s 3s/step - loss: 1.0541 - accuracy: 0.5474 - val_loss: 1.2441 - val_accuracy: 0.4318
Epoch 7/100
12/12 [==============================] - 34s 3s/step - loss: 1.0402 - accuracy: 0.5579 - val_loss: 1.2962 - val_accuracy: 0.4091
Epoch 8/100
1

Epoch 59/100
12/12 [==============================] - 34s 3s/step - loss: 0.6638 - accuracy: 0.7789 - val_loss: 1.2253 - val_accuracy: 0.5227
Epoch 60/100
12/12 [==============================] - 34s 3s/step - loss: 0.6672 - accuracy: 0.7474 - val_loss: 1.2153 - val_accuracy: 0.5455
Epoch 61/100
12/12 [==============================] - 34s 3s/step - loss: 0.6568 - accuracy: 0.7579 - val_loss: 1.2306 - val_accuracy: 0.5000
Epoch 62/100
12/12 [==============================] - 34s 3s/step - loss: 0.6534 - accuracy: 0.7474 - val_loss: 1.2073 - val_accuracy: 0.5455
Epoch 63/100
12/12 [==============================] - 34s 3s/step - loss: 0.6574 - accuracy: 0.7579 - val_loss: 1.2313 - val_accuracy: 0.5227
Epoch 64/100
12/12 [==============================] - 34s 3s/step - loss: 0.6394 - accuracy: 0.7895 - val_loss: 1.2087 - val_accuracy: 0.5227
Epoch 65/100
12/12 [==============================] - 34s 3s/step - loss: 0.6483 - accuracy: 0.7368 - val_loss: 1.2564 - val_accuracy: 0.5455
Epoch 

In [621]:
###
###

In [ ]:
#########################
#########################
#########################

In [641]:
video_path = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\train\Run\Run_0_1.mp4"
sample_video = frames_from_video_file(video_path, n_frames = 10)
sample_video.shape

(10, 224, 224, 3)

In [555]:
root_path = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips"

In [557]:
def list_videos(root_path):
    """
    Returns a list of videos and their categories.
    """
    videos = []
    for set_type in os.listdir(root_path):
        set_path = os.path.join(root_path, set_type)
        if not os.path.isdir(set_path):
            continue
        for category in os.listdir(set_path):
            category_path = os.path.join(set_path, category)
            if not os.path.isdir(category_path):
                continue
            for video in os.listdir(category_path):
                video_path = os.path.join(category_path, video)
                if not video_path.endswith(".mp4"):
                    continue
                videos.append((video_path, set_type, category))
    return videos

videos = list_videos(r"C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips")

categories = {}
for video in videos:
    category = video[2]
    if category not in categories:
        categories[category] = []
    categories[category].append(video[0])
print("Found %d videos in %d categories." % (len(videos), len(categories)))

for category, sequences in categories.items():
    summary = ", ".join(sequences[:2])
    print("%-20s %4d videos (%s, ...)" % (category, len(sequences), summary))


Found 200 videos in 6 categories.
background             71 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\background\background_13.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\background\background_31.mp4, ...)
IntCleaner             15 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntCleaner\IntCleaner_4.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntCleaner\IntCleaner_6.mp4, ...)
IntClient              75 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntClient\IntClient_15.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\IntClient\IntClient_17_1.mp4, ...)
Jolt                   19 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\Jolt\Jolt_12.mp4, C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\Jolt\Jolt_4.mp4, ...)
TS                     18 videos (C:\Users\BEE Team\Desktop\code\Summarys_labels\Sep_Clips\test\TS\TS_6

In [695]:
# Add a batch axis to the sample video.
model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

logits = i3d(model_input)['default'][0]
probabilities = tf.nn.softmax(logits)

print("Top 5 actions:")
for i in np.argsort(probabilities)[::-1][:5]:
    print(f"  {labels_fish[i]}: {probabilities[i] * 100:5.2f}%")

Top 5 actions:


IndexError: list index out of range

############################
This is in Video Classification notebook
############################

In [601]:
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

In [602]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension. 
    """
    super().__init__()
    self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [603]:
class ResidualMain(keras.layers.Layer):
  """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
  """
  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv2Plus1D(filters=filters, 
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [604]:
class Project(keras.layers.Layer):
  """
    Project certain dimensions of the tensor as the data is passed through different 
    sized filters and downsampled. 
  """
  def __init__(self, units):
    super().__init__()
    self.seq = keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [605]:
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [606]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(self.height, self.width)

  def call(self, video):
    """
      Use the einops library to resize the tensor.  

      Args:
        video: Tensor representation of the video, in the form of a set of frames.

      Return:
        A downsampled size of the video according to the new height and width it should be resized to.
    """
    # b stands for batch size, t stands for time, h stands for height, 
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos

In [607]:
input_shape = (None, 10, HEIGHT, WIDTH, 3)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))

x = layers.GlobalAveragePooling3D()(x)
x = layers.Flatten()(x)
x = layers.Dense(10)(x)

model = keras.Model(input, x)

In [608]:
frames, label = next(iter(train_ds))
model.build(frames)

In [615]:
# Visualize the model
keras.utils.plot_model(model, expand_nested=True, dpi=60, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [616]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001), 
              metrics = ['accuracy'])

In [617]:
history = model.fit(x = train_ds,
                    epochs = 50, 
                    validation_data = val_ds)

Epoch 1/50


TypeError: in user code:

    File "C:\Users\Public\new\envs\tensorflowtest\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Public\new\envs\tensorflowtest\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        It should be consistent with `x` (you cannot have Numpy inputs and
    File "C:\Users\Public\new\envs\tensorflowtest\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        `(inputs, targets, sample_weights)`.
    File "C:\Users\Public\new\envs\tensorflowtest\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        """
    File "C:\Users\Public\new\envs\tensorflowtest\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\BEETEA~1\AppData\Local\Temp\__autograph_generated_filevxom3nis.py", line 22, in tf__call
        vide